Source code of every test for the task A with a BERT model

In [1]:
import numpy as np
import pandas as pd

# Load CSV files.
#CSV task A 
def getData():
    df_train_data = pd.read_csv("data/Training_Data/subtaskA_data_all.csv")
    df_train_answers = pd.read_csv("data/Training_Data/subtaskA_answers_all.csv")

    df_train = pd.merge(df_train_data,df_train_answers,on='id', how='left').drop(['id'], axis=1)
    
    df_dev_data = pd.read_csv("data/Dev_Data/subtaskA_dev_data.csv")
    df_dev_answers = pd.read_csv("data/Dev_Data/subtaskA_gold_answers.csv")

    df_dev = pd.merge(df_dev_data,df_dev_answers,on='id', how='left').drop(['id'], axis=1)

    df_test_data = pd.read_csv("data/Test_Data/subtaskA_test_data.csv")
    df_test_answers = pd.read_csv("data/Test_Data/subtaskA_gold_answers.csv")

    df_test= pd.merge(df_test_data,df_test_answers,on='id', how='left').drop(['id'], axis=1)
    
    return df_train, df_dev, df_test

df_train_A, df_dev_A, df_test_A = getData()



Process of data frame, create subsample of it

In [2]:
def subsampleData(random):
    # subsample data 
    train = df_train_A.sample(n=1000, random_state=random)
    X_train = train[['sent0', 'sent1']]
    y_train = train['answer']

    return X_train, y_train

def subsampleDataFrequence(freq, random):
    # subsample data 
    train = df_train_A.sample(frac=freq, replace=True, random_state=random)
    X_train = train[['sent0', 'sent1']]
    y_train = train['answer']

    return X_train, y_train

# use the dev set for testing
X_test = df_dev_A[['sent0', 'sent1']]
y_test = df_dev_A['answer']

Importation of the BERT model

In [3]:
from transformers import BertModel
from bert_sklearn import BertClassifier

In [4]:
model = BertClassifier(max_seq_length=64, train_batch_size=16)
#model.num_mlp_layers = 3
model.max_seq_length = 64
model.epochs = 3
#model.learning_rate = 4e-5
                             
model

Building sklearn text classifier...


BertClassifier(max_seq_length=64, train_batch_size=16)

Methods to compute score

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [7]:
def test_performance(model, x_test, y_test):
    y_pred = model.predict(x_test)
    print(classification_report(y_pred=y_pred, y_true=y_test))
    return f1_score(y_pred=y_pred, y_true=y_test, average="binary")

Fit with different preprocess type                                                                

In [9]:
r=22
X_train, y_train = subsampleData(r)
X_train.head()

,sent0,sent1
2077,The sun rises in the East,The sun rises in the West
9207,a comet passed by the moon and a crater formed...,a comet collided the moon and a crater formed ...
6755,Driving needs leather shoes,Driving requires a driver's license
6704,Cars should pass at red lights,Cars should pass at green lights
9528,the bait is a very common dish for coastal people,the fish is a very common dish for coastal people


In [10]:
model_R20 = model.fit(X_train, y_train)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.86s/it]

Epoch 1, Train loss: 0.7218, Val loss: 0.7002, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.72s/it]

Epoch 2, Train loss: 0.6744, Val loss: 0.7302, Val accy: 49.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.77s/it]

Epoch 3, Train loss: 0.5001, Val loss: 0.8997, Val accy: 48.00%


In [12]:
f1 = test_performance(model_R20, X_test, y_test)
print(f"f1 = {f1:.3f}")

Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:39<00:00,  1.28s/it]

              precision    recall  f1-score   support

           0       0.54      0.47      0.50       518
           1       0.50      0.58      0.54       479

    accuracy                           0.52       997
   macro avg       0.52      0.52      0.52       997
weighted avg       0.52      0.52      0.52       997

f1 = 0.536


In [14]:
r=55
X_train, y_train = subsampleData(r)
X_train.head()

,sent0,sent1
2082,The horse races across the field.,The fish races across the field
679,You can drive your car to a museum,You can drive your car to a boat
2751,He checked out a book on World War II tanks at...,He bought a World War II tank at the library.
3295,I like to eat apples,I like to eat lamps
5392,I have been working as a chef for five years,I have been working as a pet for five years


In [15]:
model_R50 = model.fit(X_train, y_train)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.79s/it]

Epoch 1, Train loss: 0.7272, Val loss: 0.6960, Val accy: 53.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.78s/it]

Epoch 2, Train loss: 0.6983, Val loss: 0.6919, Val accy: 50.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.75s/it]

Epoch 3, Train loss: 0.6965, Val loss: 0.6920, Val accy: 62.00%


In [16]:
f1 = test_performance(model_R50, X_test, y_test)
print(f"f1 = {f1:.3f}")

Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:50<00:00,  1.36s/it]

              precision    recall  f1-score   support

           0       0.52      0.47      0.49       518
           1       0.48      0.53      0.50       479

    accuracy                           0.50       997
   macro avg       0.50      0.50      0.50       997
weighted avg       0.50      0.50      0.50       997

f1 = 0.503


In [52]:
r=75
X_train, y_train = subsampleData(r)
X_train.head()

,sent0,sent1
8747,Global warming shouldn't attract people's atte...,Global warming should attract people's attention
3695,All people like tea,Some people like tea
8717,People should wear sunglasses when they are sh...,People should wear glasses when they are myopic
8634,Transportation facilities belong to private pr...,Transportation facilities belong to public pro...
8955,i drink coke with a chopstick,i drink coke with a straw


In [53]:
model_R75 = model.fit(X_train, y_train)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:42<00:00,  3.28s/it]

Epoch 1, Train loss: 0.7201, Val loss: 0.6912, Val accy: 57.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:38<00:00,  2.96s/it]

Epoch 2, Train loss: 0.7019, Val loss: 0.7134, Val accy: 42.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:38<00:00,  2.93s/it]

Epoch 3, Train loss: 0.6935, Val loss: 0.7172, Val accy: 42.00%


In [63]:
f1 = test_performance(model_R75, X_test, y_test)
print(f"f1 = {f1:.3f}")

Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:56<00:00,  1.41s/it]

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       518
           1       0.00      0.00      0.00       479

    accuracy                           0.52       997
   macro avg       0.26      0.50      0.34       997
weighted avg       0.27      0.52      0.36       997

f1 = 0.000



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

In [7]:
r=42
X_train, y_train = subsampleData(r)

In [8]:
model_R42 = model.fit(X_train, y_train)
f1 = test_performance(model_R42, X_test, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Training  :   0%|                                                                               | 0/57 [00:00<?, ?it/s]C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\bert_sklearn\model\pytorch_pretrained\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.88s/it]

Epoch 1, Train loss: 0.7336, Val loss: 0.7343, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.77s/it]

Epoch 2, Train loss: 0.6791, Val loss: 0.6698, Val accy: 55.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.85s/it]

Epoch 3, Train loss: 0.4303, Val loss: 0.7001, Val accy: 66.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:55<00:00,  1.41s/it]

              precision    recall  f1-score   support

           0       0.68      0.57      0.62       518
           1       0.60      0.71      0.65       479

    accuracy                           0.64       997
   macro avg       0.64      0.64      0.64       997
weighted avg       0.64      0.64      0.64       997

f1 = 0.654


Score of models

In [17]:
r=39
X_train, y_train = subsampleData(r)

In [18]:
model_R39 = model.fit(X_train, y_train)
f1 = test_performance(model_R39, X_test, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.85s/it]

Epoch 1, Train loss: 0.7230, Val loss: 0.6978, Val accy: 47.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.85s/it]

Epoch 2, Train loss: 0.6058, Val loss: 0.5846, Val accy: 70.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:36<00:00,  2.84s/it]

Epoch 3, Train loss: 0.2148, Val loss: 0.5622, Val accy: 77.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:58<00:00,  1.43s/it]

              precision    recall  f1-score   support

           0       0.70      0.71      0.71       518
           1       0.68      0.67      0.68       479

    accuracy                           0.69       997
   macro avg       0.69      0.69      0.69       997
weighted avg       0.69      0.69      0.69       997

f1 = 0.679


In [19]:
r=38
X_train, y_train = subsampleData(r)

In [20]:
model_R38 = model.fit(X_train, y_train)
f1 = test_performance(model_R38, X_test, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:38<00:00,  2.96s/it]

Epoch 1, Train loss: 0.7222, Val loss: 0.7082, Val accy: 51.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.88s/it]

Epoch 2, Train loss: 0.6882, Val loss: 0.7175, Val accy: 52.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.87s/it]

Epoch 3, Train loss: 0.5099, Val loss: 0.7345, Val accy: 61.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:48<00:00,  1.35s/it]

              precision    recall  f1-score   support

           0       0.62      0.73      0.67       518
           1       0.64      0.53      0.58       479

    accuracy                           0.63       997
   macro avg       0.63      0.63      0.62       997
weighted avg       0.63      0.63      0.63       997

f1 = 0.577


In [25]:
r=39
X_train, y_train = subsampleDataFrequence(0.20, r)
model_R39_Bis = model.fit(X_train, y_train)
f1 = test_performance(model_R39_Bis, X_test, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 1800, validation data size: 200


Validating: 100%|██████████████████████████████████████████████████████████████████████| 25/25 [00:46<00:00,  1.86s/it]

Epoch 1, Train loss: 0.7122, Val loss: 0.6923, Val accy: 53.50%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 25/25 [00:50<00:00,  2.02s/it]

Epoch 2, Train loss: 0.6862, Val loss: 0.6815, Val accy: 50.50%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 25/25 [00:49<00:00,  1.97s/it]

Epoch 3, Train loss: 0.5076, Val loss: 0.6472, Val accy: 63.50%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:38<00:00,  1.27s/it]

              precision    recall  f1-score   support

           0       0.72      0.53      0.61       518
           1       0.60      0.78      0.68       479

    accuracy                           0.65       997
   macro avg       0.66      0.65      0.65       997
weighted avg       0.67      0.65      0.64       997

f1 = 0.681


In [9]:
model.epochs = 2
r=39
X_train, y_train = subsampleData(r)
model_R39 = model.fit(X_train, y_train)
f1 = test_performance(model_R39, X_test, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.89s/it]

Epoch 1, Train loss: 0.7209, Val loss: 0.7008, Val accy: 47.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:34<00:00,  2.63s/it]

Epoch 2, Train loss: 0.6486, Val loss: 0.6884, Val accy: 54.00%



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [02:46<00:00,  1.33s/it]

              precision    recall  f1-score   support

           0       0.58      0.68      0.63       518
           1       0.58      0.47      0.52       479

    accuracy                           0.58       997
   macro avg       0.58      0.58      0.57       997
weighted avg       0.58      0.58      0.57       997

f1 = 0.517


To save a model

In [ ]:
#save model to disk
savefile = 'BERT_TaskA.bin'
model.save(savefile)

In [18]:
from bert_sklearn import load_model
from os import path

In [20]:
savefile = path.join('models', 'BERT_A-50p.bin')
new_model = load_model(savefile)

05/28/2023 16:19:54 - INFO - bert_sklearn.model.pytorch_pretrained.modeling -   Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Loading model from models\BERT_A-50p.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...
